In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, clear_output

In [2]:
dataset= pd.read_csv("stroke prediction.csv")
dataset.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0


In [3]:
dataset.drop(columns=["id","Residence_type"],inplace=True)

In [4]:
dataset["gender"]=dataset["gender"].astype('category')
dataset["ever_married"]=dataset["ever_married"].astype('category')
# dataset["Residence_type"]=dataset["Residence_type"].astype('category')
dataset["work_type"]=dataset["work_type"].astype('category')
dataset["smoking_status"]=dataset.smoking_status.astype("category")

In [5]:
dataset_imputed=dataset.copy()

In [6]:
dataset_imputed.dtypes

gender               category
age                   float64
hypertension            int64
heart_disease           int64
ever_married         category
work_type            category
avg_glucose_level     float64
bmi                   float64
smoking_status       category
stroke                  int64
dtype: object

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()

dataset_imputed.gender=encoder.fit_transform(dataset_imputed.gender)

dataset_imputed.ever_married=encoder.fit_transform(dataset_imputed.ever_married)

dataset_imputed.work_type=encoder.fit_transform(dataset_imputed.work_type)

# dataset_imputed.Residence_type=encoder.fit_transform(dataset_imputed.Residence_type)

dataset_imputed.smoking_status=dataset_imputed.smoking_status.map({'never smoked':0,'formerly smoked':1,'smokes':2})

In [8]:
X=dataset_imputed.drop(columns="stroke",axis=1)
y=dataset_imputed["stroke"]

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=88,stratify=y)

In [9]:
from sklearn.impute import KNNImputer

imputer=KNNImputer()

X_trimputed=pd.DataFrame(imputer.fit_transform(X_train),columns=X_train.columns)

X_teimputed=pd.DataFrame(imputer.transform(X_test),columns=X_test.columns)

In [10]:
X_trimputed["smoking_status"]=X_trimputed["smoking_status"].apply(round)
X_trimputed["smoking_status"].value_counts()

X_trimputed["smoking_status"]=X_trimputed["smoking_status"].astype('category')
X_trimputed["work_type"]=X_trimputed["work_type"].apply(int)
X_trimputed["work_type"]=X_trimputed["work_type"].astype('category')

In [11]:
X_teimputed["smoking_status"]=X_teimputed["smoking_status"].apply(round)
X_teimputed["smoking_status"].value_counts()

X_teimputed["smoking_status"]=X_teimputed["smoking_status"].astype('category')
X_teimputed["work_type"]=X_teimputed["work_type"].apply(int)
X_teimputed["work_type"]=X_teimputed["work_type"].astype('category')

In [12]:
X_trimputed["gender"]=X_trimputed["gender"].astype("category")
X_trimputed[["hypertension","heart_disease","ever_married"]]=X_trimputed[["hypertension","heart_disease","ever_married"]].astype("int")

X_teimputed["gender"]=X_teimputed["gender"].astype("category")
X_teimputed[["hypertension","heart_disease","ever_married"]]=X_teimputed[["hypertension","heart_disease","ever_married"]].astype("int")

In [13]:
X_trimputed=pd.get_dummies(X_trimputed,drop_first=True)
X_teimputed=pd.get_dummies(X_teimputed,drop_first=True)

In [14]:
X_trimputed

,age,hypertension,heart_disease,ever_married,avg_glucose_level,bmi,gender_1.0,gender_2.0,work_type_1,work_type_2,work_type_3,work_type_4,smoking_status_1,smoking_status_2
0,4.0,0,0,0,136.51,17.1,1,0,0,0,0,1,0,0
1,9.0,0,0,0,90.55,27.3,1,0,0,0,0,1,0,0
2,29.0,0,0,1,108.69,29.6,1,0,0,1,0,0,0,0
3,63.0,0,0,1,84.57,31.2,1,0,0,1,0,0,1,0
4,67.0,0,0,1,70.43,26.3,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34715,62.0,0,0,1,58.90,29.1,0,0,0,1,0,0,1,0
34716,43.0,0,0,0,94.82,29.7,0,0,0,0,0,0,0,0
34717,23.0,0,0,0,98.66,28.3,0,0,0,1,0,0,0,0
34718,9.0,0,0,0,150.00,17.4,1,0,0,0,0,1,0,0


In [15]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()

scaler.fit(X_trimputed)

X_trimputed=pd.DataFrame(scaler.transform(X_trimputed),columns=X_trimputed.columns)

X_teimputed=pd.DataFrame(scaler.transform(X_teimputed),columns=X_teimputed.columns)

In [16]:
X_trimputed.head(2)

,age,hypertension,heart_disease,ever_married,avg_glucose_level,bmi,gender_1.0,gender_2.0,work_type_1,work_type_2,work_type_3,work_type_4,smoking_status_1,smoking_status_2
0,0.047852,0.0,0.0,0.0,0.345280,0.080000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.108887,0.0,0.0,0.0,0.150568,0.196571,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## SVM

### Kernel='rbf'

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report as rep
from sklearn.metrics import f1_score

In [ ]:
support=SVC(class_weight='balanced',random_state=40,probability=True)

support.fit(X_trimputed,y_train)
y_train_pred=support.predict(X_trimputed)

y_test_pred=support.predict(X_teimputed)

In [ ]:
print("Train Report")

print(rep(y_train, y_train_pred))

print("Test Report")

print(rep(y_test, y_test_pred))

In [ ]:
f1_score(y_train,y_train_pred),f1_score(y_test,y_test_pred)

In [ ]:
y_train_prob=support.predict_proba(X_trimputed)[:,1]
y_train_prob

y_test_prob=support.predict_proba(X_teimputed)[:,1]
y_test_prob


In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_train,y_train_prob),roc_auc_score(y_test,y_test_prob)

### Kernel='linear'

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report as rep
from sklearn.metrics import f1_score

In [ ]:
support=SVC(class_weight='balanced',random_state=40,kernel='linear',probability=True)

support.fit(X_trimputed,y_train)
y_train_pred=support.predict(X_trimputed)

y_test_pred=support.predict(X_teimputed)

In [ ]:
print("Train Report")

print(rep(y_train, y_train_pred))

print("Test Report")

print(rep(y_test, y_test_pred))

In [ ]:
f1_score(y_train,y_train_pred),f1_score(y_test,y_test_pred)

In [ ]:
y_train_prob=support.predict_proba(X_trimputed)[:,1]
y_train_prob

y_test_prob=support.predict_proba(X_teimputed)[:,1]
y_test_prob


In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_train,y_train_prob),roc_auc_score(y_test,y_test_prob)

### Kernel='polynomial'

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report as rep
from sklearn.metrics import f1_score

In [ ]:
support=SVC(class_weight='balanced',random_state=40,kernel='poly',probability=True)

support.fit(X_trimputed,y_train)
y_train_pred=support.predict(X_trimputed)

y_test_pred=support.predict(X_teimputed)

In [ ]:
print("Train Report")

print(rep(y_train, y_train_pred))

print("Test Report")

print(rep(y_test, y_test_pred))

In [ ]:
f1_score(y_train,y_train_pred),f1_score(y_test,y_test_pred)

In [ ]:
y_train_prob=support.predict_proba(X_trimputed)[:,1]
y_train_prob

y_test_prob=support.predict_proba(X_teimputed)[:,1]
y_test_prob


In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_train,y_train_prob),roc_auc_score(y_test,y_test_prob)

In [19]:


for i in range(2,10):
    support=SVC(class_weight='balanced',random_state=40,kernel='poly',probability=True,degree=i)

    support.fit(X_trimputed,y_train)
    y_train_pred=support.predict(X_trimputed)

    y_test_pred=support.predict(X_teimputed)

    roc_auc_score(y_train,y_train_prob),roc_auc_score(y_test,y_test_prob)

    y_train_prob=support.predict_proba(X_trimputed)[:,1]
    y_train_prob

    y_test_prob=support.predict_proba(X_teimputed)[:,1]
    y_test_prob

    print(i,f1_score(y_train,y_train_pred),f1_score(y_test,y_test_pred),roc_auc_score(y_train,y_train_prob),roc_auc_score(y_test,y_test_prob))

TypeError: __init__() got an unexpected keyword argument 'n_jobs'

## Ensembling